Hi!
I am trying to learn AutoML and decided to try out AutoML based appreoach for the competition

This is WIP and I will keep editing and improving this!

Hope you like it

# Import Libraries

In [ ]:
import os
import sys
import time
import random
import logging
import typing as tp
from pathlib import Path
from contextlib import contextmanager

from matplotlib import pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_log_error, mean_squared_error

import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoost, Pool

%matplotlib inline

# Import H20 for AutoML

In [ ]:
import h2o
print(h2o.__version__)
from h2o.automl import H2OAutoML

h2o.init(max_mem_size='16G')

# read the folders and files

In [ ]:
print(os.listdir("../input/tabular-playground-series-feb-2021"))

In [ ]:
train = h2o.import_file("../input/tabular-playground-series-feb-2021/train.csv")
train.columns


In [ ]:
test = h2o.import_file("../input/tabular-playground-series-feb-2021/test.csv")
test.columns

 Sgregate the dependent (target) and independent variables

In [ ]:
x = test.columns[1:]
y = 'target'

run the Auto ML model

In [ ]:
aml = H2OAutoML(max_runtime_secs = 50, seed = 1, project_name = "lb_frame")

aml.train(x = x, y = y, training_frame = train)

In [ ]:
lb = aml.leaderboard
lb.head() 

In [ ]:
aml.leader

In [ ]:
preds = aml.predict(test)

In [ ]:
# Get model ids for all models in the AutoML Leaderboard
model_ids = list(aml.leaderboard['model_id'].as_data_frame().iloc[:,0])

# Get the "All Models" Stacked Ensemble model
se = h2o.get_model([mid for mid in model_ids if "StackedEnsemble_AllModels" in mid][0])

# Get the Stacked Ensemble metalearner model
metalearner = h2o.get_model(se.metalearner()['name'])

In [ ]:
metalearner.coef_norm()

In [ ]:
metalearner.std_coef_plot()

In [ ]:
pred = aml.leader.predict(test)
pred.head()

In [ ]:
h2o.save_model(aml.leader, path = "./product_backorders_model_bin")

# Submit the model

In [ ]:
sample_submission = pd.read_csv('../input/tabular-playground-series-feb-2021/sample_submission.csv')


sample_submission['target'] = pred.as_data_frame().values
sample_submission.to_csv('submission.csv', index=False)

sample_submission

# Credits

https://www.kaggle.com/elvinagammed/simplest-automl-regression